VISION2CLEAN NOTEBOOK

## Standard Setup for Vision2Clean AI
This section demonstrates a standard setup for real-time waste detection using YOLO, live video feed, and logging detections with map visualization.

In [ ]:
# --- User Configuration ---
MODEL_PATH = 'yolov8n.pt'  # Change to your custom model if available
VIDEO_SOURCE = 0  # 0 for webcam, or provide video file path or IP camera URL
CAMERA_GPS = (12.9716, 77.5946)  # Example: (latitude, longitude)

In [ ]:
# Load YOLO model and set up video capture
import cv2
from ultralytics import YOLO  # Ensure YOLO is imported

model = YOLO(MODEL_PATH)
print('Model loaded:', MODEL_PATH)

cap = cv2.VideoCapture(VIDEO_SOURCE)
if not cap.isOpened():
    raise Exception(f'Cannot open video source: {VIDEO_SOURCE}')
print('Video source opened:', VIDEO_SOURCE)

In [ ]:
# Production-ready real-time detection loop
import csv
import time
import logging

# Configure logging
logging.basicConfig(filename='vision2clean.log', level=logging.INFO, 
                    format='%(asctime)s %(levelname)s:%(message)s')

def process_frame(frame, model, gps, frame_count):
    results = model(frame)
    boxes = results[0].boxes
    detections = []
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        label = model.names[cls] if hasattr(model, 'names') else str(cls)
        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
        # Log detection
        detection = {'frame': frame_count, 'label': label, 'confidence': conf, 'gps': gps}
        detections.append(detection)
        logging.info(f"Frame {frame_count}: {label} ({conf:.2f}) at {gps}")
    return frame, detections

detections_log = []
frame_count = 0
start_time = time.time()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            logging.warning('No frame received. Exiting...')
            break
        processed_frame, detections = process_frame(frame, model, CAMERA_GPS, frame_count)
        detections_log.extend(detections)
        frame_count += 1
        # Calculate and display FPS
        elapsed = time.time() - start_time
        fps = frame_count / elapsed if elapsed > 0 else 0
        cv2.putText(processed_frame, f'FPS: {fps:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
        cv2.imshow('Vision2Clean Detection', processed_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except Exception as e:
    logging.error(f"Error during detection: {e}")
finally:
    cap.release()
    cv2.destroyAllWindows()
    logging.info('Detection stopped. Resources released.')

# Print summary
print(f'Total frames processed: {frame_count}')
print(f'Total detections: {len(detections_log)}')
if len(detections_log) > 0:
    print('Sample detection:', detections_log[0])

In [ ]:
# Save detections to CSV and visualize on map
%pip install pandas

import pandas as pd
import folium

df = pd.DataFrame(detections_log)
df.to_csv('detections_log.csv', index=False)
print('Detections saved to detections_log.csv')

# Create map and add markers for each detection
detection_map = folium.Map(location=CAMERA_GPS, zoom_start=12)
for _, row in df.iterrows():
    folium.Marker(location=row['gps'], popup=f"{row['label']} ({row['confidence']:.2f})").add_to(detection_map)

detection_map

In [ ]:
# Install required packages (uncomment if running for the first time)
%pip install ultralytics opencv-python folium

In [ ]:
# Import required libraries
from ultralytics import YOLO
import cv2
import folium

print('YOLOv11/Ultralytics version:', YOLO.__module__)
print('OpenCV version:', cv2.__version__)
print('Folium version:', folium.__version__)

In [ ]:
# Load default YOLOv8n model (change to your custom model path if available)
model = YOLO('yolov8n.pt')
print('Model loaded successfully')

In [ ]:
# Minimal detection example (using the image_path variable)
results = model("D:\Documents\Portfolio.github\Vision2Clean-AI\sample.jpg")
results[0].show()  # Display detection results
print(results[0].boxes)  # Print detected bounding boxes

In [ ]:
# Minimal map visualization example
# Replace with actual coordinates and detection info
m = folium.Map(location=[12.9716, 77.5946], zoom_start=12)
folium.Marker([12.9716, 77.5946], popup='Detected Waste: Plastic').add_to(m)

m